In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
LOGDIR = os.environ['KULOKO_DIR'] 

In [ ]:
# DB libs 
from pymongo import MongoClient
sys.path.append(os.environ['COMMON_DIR'] )
from mongodb.src.mongo_handler import *

In [ ]:
#util
from util.config import ConfigManager

cm = ConfigManager(os.environ['KULOKO_INI'])

In [ ]:
# Init Logger 
logger = cm.load_log_config(os.path.join(LOGDIR,'logging.log'),log_name="KULOKO")

# Init mongo
mongo_ini=cm.load_ini_config(path=None,config_name="mongo", mode=None)

item_type = "orderbook"
mongo_db = MongoHandler(mongo_ini['LOCAL'],item_type)

In [ ]:
from pymongo import DESCENDING, ASCENDING
# mongo_db.find(sort=[('bids0',DESCENDING)])[1]
# mongo_db.find()[1]

In [ ]:
db_handler = MongoUtil(mongo_db, logger)

In [ ]:
db_handler.find_at_date("20201206")[0]

In [ ]:
MongoHandlerConditionMolts.get("filterTimeBeginWith") .format("20202")

In [ ]:
a = {"a":{"b":1}}

In [ ]:
a.update({})

In [ ]:
a

In [ ]:
mongo_db = MongoHandler(mongo_ini['LOCAL'],"trade")
print(len( [ _ for db_data in mongo_db.find()]))

In [ ]:
# orderbook
mongo_db = MongoHandler(mongo_ini['LOCAL'],"orderbook")
df_orderbook = pd.DataFrame([  db_data for db_data in mongo_db.db['orderbook'].find()])
df_orderbook.time = pd.to_datetime(df_orderbook.time, format="%Y%m%d%H%M%S%f")
df_orderbook.set_index("time",inplace=True)
del df_orderbook['_id']

In [ ]:
df_orderbook

In [ ]:
# orderbook
mongo_db = MongoHandler(mongo_ini['LOCAL'],"trade")
df_trade = pd.DataFrame([  db_data for db_data in mongo_db.db['trade'].find()])
df_trade.time = pd.to_datetime(df_trade['time'], format="%Y%m%d%H%M%S%f")
df_trade.set_index("time",inplace=True)
del df_trade['_id']

In [ ]:
df_trade

In [ ]:
df = pd.merge_asof(df_trade.reset_index(), df_orderbook.reset_index(), on="time", by="symbol")

In [ ]:
df = pd.merge_asof( df_orderbook.reset_index(),df_trade.reset_index(), on="time", by="symbol")

In [ ]:
df["spread"] = (df.bids0 -df.asks0)/df.price

In [ ]:
import holoviews as hv
%%output backend='matplotlib'
hv.extension('matplotlib')